In [1]:
import numpy as np
import pandas as pd
import torch

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import string
import re, nltk
from collections import Counter

In [2]:
df = pd.read_csv('./data/MIMIC-III-Final.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52722 entries, 0 to 52721
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  52722 non-null  int64  
 1   TEXT        52722 non-null  object 
 2   HADM_ID     52722 non-null  float64
 3   ICD9_CODE   52722 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


In [4]:
df.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,"['25013', '3371', '5849', '5780', 'V5867', '25..."
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,"['53100', '2851', '07054', '5715', '45621', '5..."
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,"['49320', '51881', '486', '20300', '2761', '78..."
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,"['56081', '5570', '9973', '486', '4019']"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,"['41401', '99604', '4142', '25000', '27800', '..."


In [5]:
len(df['HADM_ID'].unique())

52722

In [19]:
diag2idx, idx2diag = {},{}
for d in diag_keep.index:
    if d not in diag2idx:
        idx2diag[len(idx2diag)] = d
        diag2idx[d] = len(diag2idx)

In [20]:
def diag_code2idx(org_lst):
    coded_lst = []
    for c in org_lst.split(','):
        if c in diag2idx:
            coded_lst.append(diag2idx[c])
    return coded_lst

In [21]:
df['CODED_DIAG'] = df['ICD9_CODE'].apply(diag_code2idx)

In [6]:
for p in string.punctuation:
    df['ICD9_CODE'] = df['ICD9_CODE'].apply(lambda x: x.replace(p, ''))

In [7]:
df['DIAG_CODES'] = df['ICD9_CODE'].apply(lambda x: x.replace(' ', ','))

In [8]:
df.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,DIAG_CODES
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,25013 3371 5849 5780 V5867 25063 5363 4580 250...,"25013,3371,5849,5780,V5867,25063,5363,4580,250..."
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,53100 2851 07054 5715 45621 53789 4019 53550 7823,"53100,2851,07054,5715,45621,53789,4019,53550,7823"
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,49320 51881 486 20300 2761 7850 3090 V1251 V1582,"49320,51881,486,20300,2761,7850,3090,V1251,V1582"
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,56081 5570 9973 486 4019,"56081,5570,9973,486,4019"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,41401 99604 4142 25000 27800 V8535 4148 4111 V...,"41401,99604,4142,25000,27800,V8535,4148,4111,V..."


In [9]:
def high_level_code(lst):
    return [code[:3] for code in lst.split(',')]

In [10]:
df['HIGH_LVL_DIAG'] = df['DIAG_CODES'].apply(high_level_code)

In [11]:
df['HIGH_LVL_DIAG'].unique

<bound method Series.unique of 0        [250, 337, 584, 578, V58, 250, 536, 458, 250, ...
1            [531, 285, 070, 571, 456, 537, 401, 535, 782]
2            [493, 518, 486, 203, 276, 785, 309, V12, V15]
3                                [560, 557, 997, 486, 401]
4        [414, 996, 414, 250, 278, V85, 414, 411, V45, ...
                               ...                        
52717        [410, 428, 427, 427, 518, 424, 276, 511, 518]
52718        [486, 428, 518, 397, 496, 416, 585, 427, 276]
52719    [421, 746, 429, 304, 441, 442, V12, 041, 305, ...
52720    [414, 997, 997, 427, 788, 411, V45, E87, 429, ...
52721    [482, 518, 511, 584, 276, 786, 799, 414, V45, ...
Name: HIGH_LVL_DIAG, Length: 52722, dtype: object>

In [12]:
diag_codes = {}
for i, s in enumerate(df['HIGH_LVL_DIAG']):
    for w in s:
        diag_codes[w] = diag_codes.get(w, 1) + 1
diag_codes = {k: v for k, v in sorted(diag_codes.items(), key=lambda item: item[1], reverse=True)}

In [13]:
print('Diag High Level: %d' %(len(diag_codes)))

Diag High Level: 941


In [14]:
icd_groups = {
    0: '001-139',
    1: '140-239',
    2: '240-279',
    3: '280-289',
    4: '290-319',
    5: '320-389',
    6: '390-459',
    7: '460-519',
    8: '520-579',
    9: '580-629',
    10:'630-679',
    11:'680-709',
    12:'710-739',
    13:'740-759',
    14:'760-779',
    15:'780-799',
    16:'800-999',
    17:'V01-V91',
    18:'E000-E999'
}

In [15]:
def convert_hi_lvl(org_lst):
    hi_lvl = []
    for c in org_lst:
        if c.startswith('E'):
            hi_lvl.append(18)
        elif c.startswith('V'):
            hi_lvl.append(17)
        else:
            c = int(c)
            if c >= 800 and c <= 999:
                hi_lvl.append(16)
            elif c >= 780 and c <= 799:
                hi_lvl.append(15)
            elif c >= 760 and c <= 779:
                hi_lvl.append(14)
            elif c >= 740 and c <= 759:
                hi_lvl.append(13)
            elif c >= 710 and c <= 739:
                hi_lvl.append(12)
            elif c >= 680 and c <= 709:
                hi_lvl.append(11)
            elif c >= 630 and c <= 679:
                hi_lvl.append(10)
            elif c >= 580 and c <= 629:
                hi_lvl.append(9)
            elif c >= 520 and c <= 579:
                hi_lvl.append(8)
            elif c >= 460 and c <= 519:
                hi_lvl.append(7)
            elif c >= 390 and c <= 459:
                hi_lvl.append(6)
            elif c >= 320 and c <= 389:
                hi_lvl.append(5)
            elif c >= 290 and c <= 319:
                hi_lvl.append(4)
            elif c >= 280 and c <= 289:
                hi_lvl.append(3)
            elif c >= 240 and c <= 279:
                hi_lvl.append(2)
            elif c >= 140 and c <= 239:
                hi_lvl.append(1)
            elif c >= 1 and c <= 139:
                hi_lvl.append(0)
    return hi_lvl

In [16]:
df['CODED_HIGH_LVL_DIAG'] = df['HIGH_LVL_DIAG'].apply(convert_hi_lvl)

In [17]:
df.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,DIAG_CODES,HIGH_LVL_DIAG,CODED_HIGH_LVL_DIAG
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,25013 3371 5849 5780 V5867 25063 5363 4580 250...,"25013,3371,5849,5780,V5867,25063,5363,4580,250...","[250, 337, 584, 578, V58, 250, 536, 458, 250, ...","[2, 5, 9, 8, 17, 2, 8, 6, 2, 6, 9, 2, 5, 2, 11..."
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,53100 2851 07054 5715 45621 53789 4019 53550 7823,"53100,2851,07054,5715,45621,53789,4019,53550,7823","[531, 285, 070, 571, 456, 537, 401, 535, 782]","[8, 3, 0, 8, 6, 8, 6, 8, 15]"
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,49320 51881 486 20300 2761 7850 3090 V1251 V1582,"49320,51881,486,20300,2761,7850,3090,V1251,V1582","[493, 518, 486, 203, 276, 785, 309, V12, V15]","[7, 7, 7, 1, 2, 15, 4, 17, 17]"
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,56081 5570 9973 486 4019,"56081,5570,9973,486,4019","[560, 557, 997, 486, 401]","[8, 8, 16, 7, 6]"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,41401 99604 4142 25000 27800 V8535 4148 4111 V...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","[414, 996, 414, 250, 278, V85, 414, 411, V45, ...","[6, 16, 6, 2, 2, 17, 6, 6, 17, 17, 3, 6, 2, 6,..."


In [18]:
diag_keep = df['ICD9_CODE'].apply(lambda x: x.split(' ')).tolist()

In [19]:
list_sum = sum(diag_keep, [])

In [28]:
count = Counter(list_sum).most_common()[:300]

In [29]:
count

[('4019', 20053),
 ('4280', 12842),
 ('42731', 12594),
 ('41401', 12179),
 ('5849', 8907),
 ('25000', 8784),
 ('2724', 8504),
 ('51881', 7249),
 ('5990', 6442),
 ('53081', 6156),
 ('2720', 5766),
 ('2859', 5296),
 ('2449', 4788),
 ('486', 4733),
 ('2851', 4499),
 ('2762', 4358),
 ('496', 4296),
 ('99592', 3792),
 ('V5861', 3698),
 ('5070', 3592),
 ('0389', 3580),
 ('5859', 3367),
 ('40390', 3350),
 ('311', 3347),
 ('3051', 3272),
 ('412', 3203),
 ('2875', 3002),
 ('41071', 3001),
 ('2761', 2985),
 ('V4581', 2943),
 ('4240', 2878),
 ('V1582', 2741),
 ('5119', 2693),
 ('V4582', 2651),
 ('40391', 2566),
 ('V290', 2529),
 ('4241', 2517),
 ('78552', 2501),
 ('V5867', 2497),
 ('42789', 2396),
 ('32723', 2328),
 ('9971', 2313),
 ('5845', 2223),
 ('2760', 2221),
 ('7742', 2183),
 ('5180', 2137),
 ('V053', 2119),
 ('4168', 2118),
 ('49390', 2113),
 ('2767', 2113),
 ('45829', 2094),
 ('2749', 2033),
 ('4589', 1992),
 ('73300', 1910),
 ('78039', 1879),
 ('5856', 1868),
 ('5185', 1778),
 ('4271', 

In [42]:
df['CODED_HIGH_LVL_DIAG']

0        [2, 5, 9, 8, 17, 2, 8, 6, 2, 6, 9, 2, 5, 2, 11...
1                             [8, 3, 0, 8, 6, 8, 6, 8, 15]
2                           [7, 7, 7, 1, 2, 15, 4, 17, 17]
3                                         [8, 8, 16, 7, 6]
4        [6, 16, 6, 2, 2, 17, 6, 6, 17, 17, 3, 6, 2, 6,...
                               ...                        
52717                          [6, 6, 6, 6, 7, 6, 2, 7, 7]
52718                          [7, 6, 7, 6, 7, 6, 9, 6, 2]
52719                     [6, 13, 6, 4, 6, 6, 17, 0, 4, 4]
52720    [6, 16, 16, 6, 15, 6, 17, 18, 6, 6, 6, 6, 2, 7...
52721         [7, 7, 7, 9, 2, 15, 15, 6, 17, 15, 3, 17, 6]
Name: CODED_HIGH_LVL_DIAG, Length: 52722, dtype: object

In [43]:
num_labels = 19

In [58]:
def convert_label_id_to_one_hot(code_list):
    labels = np.zeros([len(code_list), num_labels])
    for idx, codes in enumerate(code_list):
        #codes = eval(codes)
        if not isinstance(codes, list):
            print("NOT a list: ", idx, codes)
        for code in codes:
            labels[idx][code] = 1
    return torch.tensor(labels)

In [59]:
code_list = df['CODED_HIGH_LVL_DIAG']
labels = convert_label_id_to_one_hot(code_list)

In [60]:
labels

tensor([[0., 0., 1.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 1., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 0., 1., 0.]], dtype=torch.float64)

In [61]:
for idx, codes in enumerate(code_list):
    print(codes)

[2, 5, 9, 8, 17, 2, 8, 6, 2, 6, 9, 2, 5, 2, 11, 17]
[8, 3, 0, 8, 6, 8, 6, 8, 15]
[7, 7, 7, 1, 2, 15, 4, 17, 17]
[8, 8, 16, 7, 6]
[6, 16, 6, 2, 2, 17, 6, 6, 17, 17, 3, 6, 2, 6, 6, 17, 17, 17]
[1, 1, 1, 1, 2]
[16, 16, 16, 7, 16, 3, 16, 16, 16, 16, 4, 16, 16, 18]
[6, 16, 6, 6, 2, 17, 17, 17, 17, 17, 18]
[7, 7, 2, 7, 13, 5, 2, 6, 4, 8, 17]
[1, 11, 6, 6, 5, 12, 16, 2, 13, 11, 17, 7, 11, 6, 6, 17, 9, 2, 6, 17, 2, 5, 2, 6, 7, 6, 2, 8, 6]
[6, 13, 6, 15]
[9, 5, 2, 8, 9, 0, 6, 9, 5, 4, 0, 8, 5, 5, 6, 5, 8, 17]
[18, 18, 6, 0, 0, 8, 17, 3, 15, 16, 16, 8, 4, 4, 4, 4, 6, 15, 3, 9, 6, 7, 15, 7, 8, 9, 16, 18, 18]
[6, 6, 16, 7, 7, 17, 6, 6, 18, 6, 2]
[17, 14, 16, 14, 14, 17, 18, 17]
[8, 0, 7, 8, 16, 6, 3, 16, 18]
[14, 14, 17, 14]
[6, 7, 6, 2, 8, 3]
[6, 6, 6, 6, 8, 6, 6, 12, 8]
[6, 2, 2, 6, 6, 11, 6, 17]
[7, 6, 5, 7, 15, 6, 9, 16, 2, 5, 6, 2, 7, 16, 17, 7, 4, 6, 9, 2, 18, 5, 18, 2, 18, 6]
[6, 6, 16, 6, 2, 6, 6, 18, 8, 6, 9, 6, 4, 6, 3, 6, 6]
[1, 9, 6, 7, 6, 11, 3, 3, 5, 17, 0, 1, 3, 3, 6, 15, 6, 6, 15, 

[6, 6, 6, 7, 3, 0, 18, 2, 6]
[9, 5, 7, 4, 3, 0, 16, 2, 6, 2, 12, 17, 18, 15]
[7, 7, 2, 1, 1, 1, 1, 1, 3, 3, 17, 4]
[6]
[5, 7, 9, 7, 1, 1, 1, 1, 0, 12, 6, 2, 2, 8, 15, 3, 17]
[16, 15, 18, 18]
[7, 7, 9, 6]
[8, 8, 8, 2, 15, 3, 15, 3, 4, 4, 6, 0, 0, 7, 8, 9, 8, 7, 5, 6, 2, 2, 9, 15]
[0, 9, 2, 6, 6, 6, 2, 2, 6, 5, 17, 2, 6]
[1, 6, 7, 0, 2, 6, 3, 2, 6]
[6, 9, 2, 7, 7, 8, 2, 2, 15]
[8, 8, 8, 9, 3, 7, 7, 15, 0, 16, 3, 2, 6, 4, 16, 16, 0, 2, 3, 3, 6, 12, 2, 6, 18, 18, 17, 17]
[16, 16, 18, 16]
[9, 6, 9, 2, 0, 6, 4, 4, 15, 2, 17, 3]
[6, 13, 9, 9, 12, 6, 7, 12, 6, 6, 6, 15, 0, 2, 6]
[1, 2, 4, 9, 3, 6, 8, 2, 6, 7, 3, 17, 0, 0, 11, 11, 3]
[6, 6, 16, 16, 15, 6, 6, 2, 17]
[6, 9, 2, 2, 2, 2, 6, 2, 2]
[7, 7, 2, 4, 2, 0, 7, 7, 5, 18, 6, 17, 15, 4, 18, 18, 9, 2, 13, 15, 17]
[0, 0, 3, 9, 16, 3, 6, 5, 6, 15]
[6, 9, 6, 9, 13, 6, 6, 2, 6, 5, 12, 4, 17]
[6, 7, 7, 6, 6, 11, 6, 17, 17, 2, 8, 2, 6, 2, 17, 17, 17]
[16, 15, 7, 8, 9, 5, 11, 11, 7, 16, 18, 18, 12, 12, 12, 2, 6, 4, 4, 17, 17]
[6, 5, 16, 2, 17, 6, 2]
[

[15, 16, 18, 17, 6, 2, 2, 5, 12, 12, 4, 6, 6, 7, 17, 17]
[7, 7, 6, 15, 0, 6, 2, 7, 8]
[7, 6, 9, 15, 3, 6, 6, 1, 0, 3, 17, 17, 6, 17]
[1, 8, 8, 8, 8, 0, 16, 7, 15]
[1, 7, 1, 17, 2, 1, 15, 2, 17, 2, 2, 17, 17, 17]
[1, 16, 6, 7, 7, 8, 15, 15]
[16, 18, 12]
[7, 7, 9, 9, 6, 2, 0, 17]
[16, 7, 0, 3, 9, 16, 16, 18, 16, 4]
[9, 2, 17, 2, 17, 5, 6]
[2, 4, 2, 15, 2, 7, 2, 4, 4, 2, 2, 8, 8, 6, 4, 3, 3, 6]
[8, 8, 16, 2, 2, 12, 8]
[16, 6, 6, 6, 9, 11, 6, 7, 6, 2]
[8, 6, 7, 6, 1, 6, 9, 8, 6, 7]
[16, 5, 6, 16, 6, 5, 18, 18, 7, 6, 2, 4, 4]
[1, 5, 3, 1, 15, 5, 9, 17, 17, 12, 7, 17, 17]
[16, 16, 6, 7, 16, 5, 2, 6, 18]
[6, 7, 6, 9, 6, 2, 9, 3, 6, 6, 6, 3]
[1, 6, 2, 3]
[7, 5, 15, 8, 1, 15, 2, 7, 12, 16, 17, 2, 15, 8, 6, 15, 12, 4, 17, 17, 17, 18, 17, 17, 17]
[1, 1, 8, 7, 7, 6, 12, 6, 2]
[6, 6, 6, 6, 6, 15, 6]
[6, 7, 5, 7, 16, 2, 5, 5, 15, 18]
[17, 14, 8, 14, 14, 17, 17, 17, 17]
[11, 0, 16, 7, 7, 8, 8, 2, 9, 17, 8, 2, 6, 15, 5, 17, 4, 6, 8]
[3, 7, 9, 1, 5, 6, 9, 6, 11, 7, 6, 6, 1, 6, 17, 3, 0, 17, 0, 6, 6, 0,

[0, 7, 15, 7, 3, 6, 6, 6, 16, 6, 2, 1, 6, 2, 17]
[16, 16, 16, 18, 18, 3, 12]
[6, 6, 6, 7, 7, 12, 16, 6, 6]
[16, 6, 6, 6, 8, 6]
[6, 6, 6, 2, 6, 2, 4]
[16, 16, 16, 16, 18]
[2, 9, 6, 9, 2, 7, 2]
[6, 6, 7, 6, 6, 17, 17, 6]
[6, 6, 6, 15, 2, 9, 0, 17, 6]
[9, 9, 15, 3, 15, 0, 6, 2, 2]
[7, 16, 9, 2, 9, 9, 6, 6, 6, 7, 0, 3, 15, 5, 2, 1, 3, 17, 7, 18, 17]
[7, 2, 3, 7, 15, 1, 16, 16, 18, 9, 15, 18, 2, 15, 2, 4, 17]
[6, 2, 8]
[6, 6, 3, 2, 9, 11, 3, 2, 5, 2, 5, 17, 2, 16, 18, 0, 12, 16]
[7, 1, 1, 15, 1, 8, 17, 17]
[6, 6, 6, 6, 6, 7, 17, 6, 2, 6, 6, 2, 7, 17, 17, 11, 3]
[7, 7, 8, 5, 4, 4]
[17, 14, 14, 14, 14, 17, 17]
[6, 6, 6, 18, 6, 2, 17, 17, 2, 2, 12, 9, 2, 6, 3, 9, 15]
[4, 2, 15, 8, 4, 3, 16, 18, 5, 8, 17]
[16, 8, 7, 18, 7, 7, 16, 1, 17]
[17, 14, 17, 17]
[8, 6, 3, 8, 0, 7, 0, 0, 8]
[8, 4, 2, 2, 2, 7]
[0, 15, 9, 16, 8, 8, 6, 2, 0, 17]
[1, 6, 2, 2, 6]
[16, 18, 6]
[6, 2, 6, 2, 6, 17]
[6, 8, 6, 2, 2, 15, 6, 7, 2, 6, 6, 2, 15, 3, 2, 3, 8, 12, 17, 17, 6]
[16, 4, 4, 9, 4, 16, 8, 17, 18, 4, 4, 18, 17, 1

[6, 9, 16, 3, 18, 6, 9, 6, 17, 15, 7, 17, 3, 2, 18, 5]
[17, 14, 14, 14, 14, 17, 14, 17]
[15, 6, 7, 16, 6, 6, 18, 2, 7, 6, 12, 8, 2]
[6, 6, 9, 6, 1, 2, 6, 9, 6, 9, 3, 17, 6, 17, 17, 6]
[9, 7, 8, 15, 2, 2, 2, 6, 6, 2, 6, 2]
[7, 7, 7, 9, 8, 8, 16, 0, 8, 5, 15, 8, 6, 4, 2, 3, 12, 5, 5, 12, 4, 17, 18]
[16, 16, 8, 8, 8, 9, 7, 7, 7, 2, 18, 12, 6, 17]
[8, 2, 6, 4, 17, 17, 17, 17]
[4, 5, 17, 18, 4, 4]
[16, 7, 4, 4, 15, 16, 16, 18, 16, 16, 16, 4, 8]
[8, 15, 1, 9, 0, 8, 3, 17, 15, 4]
[16, 7, 18, 6, 2, 5, 2, 5, 12, 7]
[17, 14, 14, 14, 14, 14, 14, 14, 14, 14, 11]
[6, 16, 7, 8, 9, 16, 4, 18, 17]
[6, 6]
[6, 6, 6, 16, 6, 6, 2, 6, 2]
[15, 15, 2, 0, 5, 2, 6, 8, 2, 6, 17, 5, 4]
[16, 6, 15, 4, 11, 18, 18]
[15, 9, 15, 18, 3, 3, 6, 17, 6, 17, 6, 8, 7, 2, 2, 12, 17, 17, 17, 17, 17, 17]
[1, 7, 1, 1, 7, 12, 7, 1, 0, 16, 3, 17, 15]
[16, 1, 9, 6, 6, 3, 18, 1, 2, 6, 4]
[8, 3, 9, 6, 8, 1, 8, 2, 2]
[7, 7, 9, 16, 18, 6, 2, 4, 5, 17]
[5, 5, 15, 7, 0, 7, 2, 8, 15, 3, 18]
[8, 0, 1, 16, 7, 7, 9, 0, 16, 0, 16, 17, 8, 6, 

...

[1, 1, 7, 17, 17]
[16, 7, 7, 4, 4, 18, 6, 12, 4]
[6, 7, 16, 7, 7, 16, 6, 3, 8]
[6, 6, 6, 6, 16, 9, 7, 7, 18]
[6, 6, 6, 6, 2, 6]
[0, 15, 7, 9, 1, 16, 6, 7, 2, 15, 5]
[2, 17, 2, 6, 6, 15, 2, 9, 2, 6, 6, 6, 9, 9, 16, 6, 3]
[2, 2, 9, 7, 16, 6, 6, 9, 0, 6, 6, 15, 3, 3, 15, 3, 2, 4, 18, 17, 17, 17, 17]
[6, 6, 6, 2, 6, 12, 2, 12, 6, 6]
[17, 14, 14, 14, 17]
[8, 6, 2, 2, 4, 2, 15, 6, 6, 17, 6, 12, 17]
[2, 9, 2, 0, 7, 2, 8, 8, 8, 2, 5, 2, 9, 17, 3, 15, 3, 2, 9, 6, 9, 15, 3, 2, 2, 6, 4, 18]
[7, 16, 6, 6, 15, 9, 3, 4, 15, 15, 17, 17, 17, 17, 0, 7, 15, 6, 7, 9, 7, 9, 4, 2, 9, 2]
[6, 6, 6, 16, 7, 8, 18, 2, 6]
[2, 2, 17, 15, 7]
[6, 6, 6, 16, 6, 6, 4, 18, 18, 7, 4, 5, 12, 5, 6, 17, 2]
[6, 9, 6, 6, 11, 6, 15, 6, 9, 2, 6, 17, 17, 6, 7, 6, 15]
[8, 8, 17, 6, 6, 7, 11, 18, 18, 3, 6]
[8, 8, 8, 8, 6, 3, 15, 6, 6]
[0, 8, 9, 15, 8, 2, 16, 6, 2, 3, 8, 6, 6, 2, 2, 2, 12, 17, 17, 17, 17, 17]
[6, 6, 7, 6, 6, 4, 15, 2, 3, 2, 6, 6]
[6, 6, 6, 6, 6, 17, 11, 6, 2, 2, 12, 17, 17, 6, 17]
[0, 9, 3]
[1, 11, 1, 11, 17, 6, 2

[6, 6, 6, 6, 7, 6, 2, 2, 6, 17, 17, 4, 17, 17]
[5, 6, 11, 2, 17, 5]
[7, 7, 9, 5, 6, 2, 6, 15, 7, 2]
[8, 0, 7, 9, 16, 5, 11, 16, 0, 3, 3, 3, 3, 7, 8, 0, 0, 16, 6, 2]
[17, 14, 14, 14, 14, 17, 17]
[6, 6, 0, 6, 6, 6, 6, 5]
[6, 9, 11, 3, 15, 12, 13, 6, 6]
[6, 16, 3, 6, 6, 2, 8, 17, 17, 18]
[6, 6, 6, 2, 17, 17, 5]
[1, 6, 9, 7, 1, 1, 1, 1, 8, 8, 16, 9, 18, 1, 6, 3]
[17, 17, 17, 1, 15, 1, 1, 15, 2, 6, 8, 3, 5, 11, 11, 4, 12, 4, 4]
[0, 7, 7, 9, 6, 2, 6, 6, 7, 2, 6, 3, 2, 16, 17, 5, 5, 17, 3, 4, 2, 5, 12, 12, 4, 15, 17, 17]
[6, 7, 9, 6, 6, 2, 8, 5, 4, 0, 7, 2, 6, 9, 2, 5, 6, 17, 17, 17]
[8, 6, 3, 6, 2, 17, 6, 17, 6, 6, 15, 8, 8, 6, 17, 17, 18]
[8, 8, 16, 0, 8, 8, 16, 16, 7, 8, 9, 6, 8, 2, 12, 8, 2, 18, 18, 15, 0]
[6, 6, 2]
[13, 6, 6, 6]
[6, 6, 12, 6, 15, 6, 17, 17, 17, 18, 3]
[16, 16, 16, 16, 6, 15, 16, 18]
[6, 11, 2, 9, 17, 17, 17]
[16, 11, 16, 16, 16, 16, 16, 6, 11, 18, 18, 6, 6, 6, 4, 8, 6, 16, 17]
[7, 6, 6, 9, 6, 15, 2, 6, 2]
[1, 6, 6, 3, 16, 11, 9, 6, 17, 17, 6, 15, 18, 15, 18]
[16, 7, 16, 

[0, 7, 7, 5, 9, 4, 7, 7, 7, 8, 16, 6, 15, 17, 11, 11, 3]
[16, 6, 18, 17, 7, 9, 2, 6, 17]
[16, 0, 18, 12, 2, 6, 17, 2, 15]
[6, 17, 15, 7, 6, 6, 6, 17, 4, 15, 6, 2, 6, 8, 6, 2, 12, 4, 17, 17, 17, 17]
[17, 14, 14, 14, 14, 14, 14, 17, 17]
[9, 15, 6, 6, 15, 6, 2, 2, 6, 17, 17]
[16, 16, 7, 3, 12, 17, 17, 5]
[10, 10, 10, 10, 10, 16, 11, 10, 17]
[7, 0, 2, 1, 2, 8, 17, 4, 2, 15, 8, 3, 16, 18, 17, 3, 18, 18]
[1, 6, 5, 7, 9, 7, 2, 6, 9, 3, 3]
[9, 16, 0, 2, 0, 2, 6, 6, 11, 0, 5, 3, 6, 12, 12]
[16, 7, 6, 6, 2, 7, 9, 6, 18, 17, 1, 17]
[7, 16, 7, 7, 7, 6, 6, 7, 7]
[6, 2, 8, 5, 6, 9]
[6, 16, 6, 16, 8, 6, 1, 6]
[8, 6, 8, 8, 8, 6]
[1, 16, 16, 8, 17, 3, 6, 6, 6, 17, 17]
[16, 4, 4, 4, 4, 18, 18, 16, 18, 8, 4, 16, 18, 3]
[17, 14, 17, 17]
[0, 6, 7, 6, 2, 16, 8, 8, 15, 17, 7, 17, 6, 9, 17, 6, 16, 18, 18, 15, 15, 6, 6, 16, 2, 2, 17]
[16, 7, 5, 7, 5, 15, 15, 7, 9, 18, 1, 3, 6, 2, 2, 16, 18, 0, 17, 15, 17, 6, 7, 17, 6, 15]
[7, 2, 7, 6, 7, 4, 4, 4]
[0, 6, 7, 17, 11, 7, 12, 2, 2]
[6, 6, 6, 7, 16, 7, 6, 1, 18]
[0,

[16, 5, 5, 16, 7, 5, 16, 6, 5, 2, 6, 4, 4, 4, 4, 18]
[6, 9, 6, 9, 6, 7, 6, 6, 6, 2, 3, 15, 2]
[6, 6, 9, 0, 2]
[16, 16, 7, 7, 4, 3, 9, 15, 8, 15, 6, 3, 6, 5, 2, 18, 18, 18]
[6, 9, 3, 6, 6, 6, 17, 2, 17]
[7, 7, 17, 2, 5, 2]
[8, 6, 6, 7, 8, 2, 6, 6, 6, 7, 6, 6, 2, 15]
[12, 6, 7, 8, 6, 18, 18, 15, 4, 4]
[0, 15, 7, 9, 8, 8, 2, 16, 6, 2, 5, 2, 9, 2, 2, 6, 17, 17, 17]
[6, 7, 6]
[6, 9, 6, 2]
[1, 5, 6, 6, 6, 17]
[0, 9, 6, 2, 2, 6, 6, 6, 17, 17, 17, 17]
[16, 16, 16, 16, 16, 16, 16, 16, 16, 18, 18, 4]
[1, 7]
[8, 8, 8, 0, 16, 15, 3, 2, 4, 9, 2, 8, 4, 4, 8, 2, 2, 12, 11, 11, 3, 8, 3, 15, 17]
[7, 6, 6, 9, 15, 9, 2, 9, 6, 7, 15]
[6, 6, 2]
[8, 0, 9, 8, 6, 16, 8, 6, 6, 6, 6, 7, 17, 6, 9, 17]
[16, 6, 6, 6, 17, 6, 12, 6, 6, 2, 17, 17, 12, 8, 17, 5, 12, 15]
[6, 6, 6, 6, 6, 6, 6, 6, 2, 9, 8]
[6, 7, 16, 16, 6, 5, 15]
[1, 16, 6, 8, 9, 6, 1, 1, 1, 8, 2, 6, 3]
[16, 17, 17, 9, 6, 5, 17, 6, 18, 5, 2, 6, 9, 2, 8, 2, 5, 17, 17]
[17, 13, 17]
[6, 15, 7, 16, 7, 16, 6, 6, 6]
[12, 7, 5, 13, 5, 12, 18, 5, 17, 17]
[6, 8,

[16, 16, 8, 16, 8, 6, 16, 2, 2, 18, 18, 6, 3, 17, 6, 4]
[16, 6, 6, 7, 7, 6, 7, 12, 18]
[16, 6, 7, 16, 5, 15, 15, 5, 6, 6, 6, 2, 8, 15, 12, 17, 18]
[6, 7, 11, 7, 16, 16, 7, 3, 11, 11, 6, 2, 15, 18]
[6, 6, 7, 6, 9, 6, 6, 6, 17, 17]
[0, 8, 7, 6, 6, 9, 9, 3, 3, 15, 6, 15, 6, 2, 17, 16, 8, 17, 18]
[6, 6, 6, 16, 8, 6, 7, 7, 6, 2, 17, 17, 17]
[8, 7, 2, 7, 9, 4, 8, 16, 18, 4, 6, 8, 17, 0]
[1, 5, 7, 1, 1, 5, 1, 1, 1, 7, 17, 9, 15, 3]
[7, 6, 2, 2, 6, 2, 2, 5, 3]
[16, 16, 7, 7, 6, 7, 6, 16, 11, 16, 16, 18, 2, 2, 6, 8, 8, 13]
[6, 6, 6, 2, 2, 9, 6, 6, 17, 8, 17, 2]
[3, 7, 6, 6, 7]
[2, 2, 11, 7, 3, 17, 18, 15, 5, 15, 3, 2, 11, 11, 11, 6, 15, 12, 2, 11, 15, 17]
[7, 8, 5, 4, 15, 4, 15, 17]
[6, 3, 6]
[12, 6, 11, 16, 16, 18, 15, 3, 2]
[16, 5, 16, 5, 16, 16, 18, 16, 4, 15, 16, 4]
[6, 6, 16, 16, 11, 9, 6, 6, 18, 3, 2, 6, 17, 2, 17]
[8, 16, 6, 6, 9, 16, 7, 2, 9]
[6, 6, 7, 2, 6, 15, 6, 2, 8, 15, 17, 4, 15, 17, 17]
[1, 10, 15]
[6, 6, 2, 3, 2, 7, 6, 12, 6, 6, 6, 17]
[6, 3, 12, 6, 2, 15, 17]
[6, 9, 16, 6, 16, 

In [122]:
def remove_stopwords(text): 
        stop_words = set(stopwords.words("english")) 
        word_tokens = word_tokenize(text) 
        filtered_text = [word for word in word_tokens if word not in stop_words] 
        return filtered_text 

In [116]:
def preprocess(note):
    note = note.replace('\n',' ')
    note = note.replace('w/', 'with')
    note = note.lower() #lower case
    note = re.sub(r'\d+', '', note) #remove numbers
    note = note.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    note = " ".join(note.split())
    note = remove_stopwords(note)
    return note

In [123]:
df['CLEAN_WORDS'] = df['TEXT'].apply(preprocess)

In [125]:
df['CLEAN_WORDS'][0]

['admission',
 'date',
 'discharge',
 'date',
 'date',
 'birth',
 'sex',
 'f',
 'service',
 'medicine',
 'allergies',
 'levaquin',
 'attendingfirst',
 'name',
 'lf',
 'chief',
 'complaint',
 'nausea',
 'vomiting',
 'major',
 'surgical',
 'invasive',
 'procedure',
 'none',
 'history',
 'present',
 'illness',
 'f',
 'poorly',
 'controlled',
 'type',
 'diabetes',
 'mellitus',
 'neuropathy',
 'nephropathy',
 'htn',
 'gastroparesis',
 'ckd',
 'retinopathy',
 'recently',
 'hospitalized',
 'orthostatic',
 'hypotension',
 'autonomic',
 'neuropathy',
 'date',
 'range',
 'dka',
 'hospitalizations',
 'returning',
 'history',
 'worsening',
 'nausea',
 'vomiting',
 'coffeeground',
 'emesis',
 'chills',
 'dyspnea',
 'exertion',
 'last',
 'week',
 'fall',
 'hit',
 'right',
 'face',
 'also',
 'day',
 'diarrhea',
 'resolved',
 'early',
 'last',
 'week',
 'found',
 'dka',
 'ag',
 'bicarb',
 'ed',
 'inital',
 'vitals',
 'ra',
 'k',
 'hco',
 'anion',
 'gap',
 'cr',
 'baseline',
 'rd',
 'l',
 'ns',
 'insul